In [3]:
# This script was used on 2023-11-13 to convert "Non Grocery Packet Timeout/Delivered" statuses to 
# their corresponding parts: Pads, Diapers, and Soap & Shower Products
# It only added the corresponding Timeout/Delivered Tag based on what was present in the "Requests" column
# So if a record indicated that someone requested Pads and had a status of "Non Grocery Packet Delivered" 
# then it only added "Pads Delivered". After this script ran, we deleted the 
# "Non Grocery Packet Timeout/Delivered" statuses from the field options.

In [ ]:
from bam_core.lib.airtable import Airtable
import time

airtable = Airtable()

records = airtable.assistance_requests.all(
            fields=["Phone Number", "Essential Goods Requests?", "Essential Goods Requests Status"],
            view="Non Grocery Packet View To Clean",
)

request_status_mapping = {
    "Pañales de adultos / Adult Diapers / 成人纸尿裤": {
        "timeout": "Adult Diapers Timeout",
        "delivered": "Adult Diapers Delivered",
    },
    "Pañales / Baby Diapers / 婴儿纸尿裤": {
        "timeout": "Baby Diapers Timeout",
        "delivered": "Baby Diapers Delivered",
    },
    "Productos Femenino - Toallitas / Feminine Products - Pads / 卫生巾": {
        "timeout": "Pads Timeout",
        "delivered": "Pads Delivered",
    },
    "Jabón & Productos de baño / Soap & Shower Products / 肥皂和淋浴产品": {
        "timeout": "Soap & Shower Products Timeout",
        "delivered": "Soap & Shower Products Delivered"
    }
}

for record in records:
    id = record['id']
    new_statuses = []
    requests = record.get('fields', {}).get('Essential Goods Requests?', [])
    old_statuses = record.get('fields', {}).get('Essential Goods Requests Status', [])

    # lookup new statuses based on what was requested
    for request in requests:
        if request in request_status_mapping:
            if 'Non Grocery Packet Delivered' in old_statuses:
                new_statuses.append(request_status_mapping[request]["delivered"])
            elif 'Non Grocery Packet Timeout' in old_statuses:
                new_statuses.append(request_status_mapping[request]["timeout"])
    # remove any 'Non Grocery Packet' statuses
    for status in old_statuses:
        if not status.startswith('Non Grocery Packet'):
            new_statuses.append(status)
    new_statuses = list(set(new_statuses))
    print(f"Changing {id}:{requests} from {old_statuses} to {new_statuses}")
    # uncomment the following line to actually update records
    # airtable.assistance_requests.update(id, {'Essential Goods Requests Status': new_statuses})
    time.sleep(0.1)
